In [ ]:
# DO NOT REMOVE

# Load hypocentral depth distribution from .csv files

In [ ]:
%%html
<script>
    var code_show=true; //true -> hide code at first

    function code_toggle() {
        $('div.prompt').hide(); // always hide prompt

        if (code_show){
            $('div.input').hide();
        } else {
            $('div.input').show();
        }
        
        code_show = !code_show
    }
    $( document ).ready(code_toggle);
</script>
<p style="font-size:60%;">
<a href="javascript:code_toggle()">[Toggle Code]</a>
<a target="_blank" href="./../project/project_set_params_gui.ipynb#">[Set params]</a>
</p>

In [ ]:
%matplotlib inline
import os
import re
import sys
import h5py
import numpy
import pickle
import matplotlib.pyplot as plt
from oqmbt.oqt_project import OQtProject

In [ ]:
project_pickle_filename = os.environ.get('OQMBT_PROJECT')
oqtkp = OQtProject.load_from_file(project_pickle_filename)
model_id = oqtkp.active_model_id
model = oqtkp.models[model_id]
# hdf5 files
hypo_depths_hdf5_filename = os.path.join(oqtkp.directory, oqtkp.hypo_depths_hdf5_filename)
# set source ID
try:
    area_source_ids_list = getattr(oqtkp,'active_source_id')
except:
    print('Active source ID not defined in the OQMBT project')
    area_source_ids_list = ['10']
print('Processing area source with ID:', area_source_ids_list)
src_id = area_source_ids_list[0]

In [ ]:
aa = re.sub('\.hdf5', '', os.path.basename(hypo_depths_hdf5_filename))
csv_filename = '{:s}-{:s}-{:s}.csv'.format(aa, model_id, src_id)
path = os.path.join(oqtkp.directory, 'hypo_depths')
infile = os.path.join(path, csv_filename)
print('Input file:', infile)

In [ ]:
#
# loading csv file
from decimal import *
getcontext().prec = 10

wei = []
dep = []
print ('Reading {:s}'.format(infile))
for line in open(infile, 'r'):
    if not re.search('[a-z]', line): 
        aa = re.split('\,', line)
        dep.append(float(aa[0]))
        wei.append(float(aa[1]))
nwei = []
for w in wei:
    tmp = Decimal('{:.8f}'.format(w))
    nwei.append(tmp)    
wei = numpy.array(wei)
dep = numpy.array(dep)

smm = sum(wei)
wei = wei / smm

In [ ]:
#
# open the hdf5 containing the hypocentral depth information
hypo_dist_filename = os.path.join(oqtkp.directory, model.hypo_dist_filename)
fhdf5 = h5py.File(hypo_dist_filename, 'a')
#
# cleaning
if src_id in fhdf5:
    del fhdf5[src_id]
#
# updating the dataset for the current area source
x = numpy.zeros(len(wei), dtype=[('depth','f4'),('wei', 'f4')])
x['wei'] = wei
x['depth'] = dep
dset = fhdf5.create_dataset(src_id, data=x)
fhdf5.close()

print('{:s} updated - src {:s}'.format(hypo_dist_filename, src_id))